# MobileNet 18→13 Class Conversion

This notebook converts the trained 18-class MobileNet model to a proper 13-class model by:
1. Loading the original 18-class model
2. Extracting the feature backbone
3. Adding a new 13-class classification head
4. Fine-tuning on 13-class data
5. Creating the production model with correct preprocessing

In [ ]:
# Setup and imports
import tensorflow as tf
import keras
import numpy as np
import os
import json
from PIL import Image

# GPU setup
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

print(f"TensorFlow: {tf.__version__}")
print(f"Keras: {keras.__version__}")
print(f"Using device: {'GPU' if tf.config.list_physical_devices('GPU') else 'CPU'}")

In [ ]:
# Load the original 18-class model
original_model = keras.models.load_model("mobilnet_masterclass.keras")

original_model.summary()

print(f"\nCurrent model :{original_model.layers[-1].units} output classes")

In [ ]:
# Extract backbone and create 13-class model
dense_layers = [(i, layer) for i, layer in enumerate(original_model.layers) 
                if isinstance(layer, keras.layers.Dense)]

backbone_end_idx = dense_layers[-2][0]
backbone_end_layer = dense_layers[-2][1]

# Rebuild backbone
backbone_layers = original_model.layers[:backbone_end_idx + 1]
new_backbone = keras.Sequential(name="backbone_13class")

# Copy each layer to preserve weights
for i, layer in enumerate(backbone_layers):
    config = layer.get_config()
    weights = layer.get_weights()
    
    new_layer = layer.__class__.from_config(config)
    new_backbone.add(new_layer)
    
    if weights:
        if i == 0:
            dummy_input = np.random.random((1, 224, 224, 3))
            new_backbone(dummy_input)
        new_layer.set_weights(weights)

# Create 13-class model
model_13class = keras.Sequential([
    new_backbone,
    keras.layers.Dense(13, activation="softmax", name="classification_13class")
], name="mobilenet_13class")

# Build the model
dummy_input = np.random.random((1, 224, 224, 3))
_ = model_13class(dummy_input)

print(f"\nNew model with {model_13class.output_shape} output")

In [ ]:
# Load 13-class dataset and fine-tune
print("Loading 13-class dataset...")
real_ds = keras.preprocessing.image_dataset_from_directory(
    "../../data/dataset_no_oat_downsample",
    image_size=(224, 224),
    batch_size=32,
)

train_size = int(0.8 * len(real_ds))
real_train_ds = real_ds.take(train_size)
real_val_ds = real_ds.skip(train_size)

data_dir = "../../data/dataset_no_oat_downsample"
class_names = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
print(f"Classes ({len(class_names)}): {class_names}")

model_13class.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model_13class.fit(
    real_train_ds,
    epochs=30,
    validation_data=real_val_ds,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    ]
)

# Evaluate
val_loss, val_accuracy = model_13class.evaluate(real_val_ds, verbose=0)
print(f"Validation accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")

In [ ]:
# Create production model with preprocessing

production_model = keras.Sequential([
    keras.layers.Resizing(224, 224, interpolation="bilinear", name="resize_to_224"),
    # No Rescaling layer : model was trained on [0,255] pixel values
    model_13class,
], name="wildlens_13class_production")

production_model.summary()

# Save models
model_13class.save("mobilenet_13class_corrected.keras")
production_model.save("wildlens_multiclassifier.keras")

print("\nModels saved:")
print("   - mobilenet_13class_corrected.keras (13-class model)")
print("   - wildlens_multiclassifier.keras (production model)")

In [ ]:
# Quick validation test

# Simple inference function
def predict_track(image_path):
    img = Image.open(image_path).convert('RGB')
    img_array = np.array(img, dtype=np.uint8)
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = production_model.predict(img_array, verbose=0)
    predicted_idx = np.argmax(predictions[0])
    confidence = predictions[0][predicted_idx]
    predicted_class = class_names[predicted_idx]
    
    return predicted_class, confidence

# Test with a few sample images
test_images = []
for class_name in class_names[:3]:  # Test first 3 classes
    class_dir = f"../../data/dataset_no_oat_downsample/{class_name}"
    if os.path.exists(class_dir):
        images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if images:
            test_images.append(os.path.join(class_dir, images[0]))

print(f"\nTesting {len(test_images)} sample images:")
correct = 0
for img_path in test_images:
    predicted_class, confidence = predict_track(img_path)
    true_class = os.path.basename(os.path.dirname(img_path))
    is_correct = predicted_class == true_class
    if is_correct:
        correct += 1
    
    print(f"{os.path.basename(img_path)}: {true_class} -> {predicted_class} ({confidence:.3f}) {'Correct' if is_correct else 'Incorrect'}")

print(f"\nResults: {correct}/{len(test_images)} correct ({correct/len(test_images)*100:.1f}%)")